<a href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/ml-pipelines-node-classification.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning pipelines: Node classification

This notebook shows the usage of GDS machine learning pipelines with the Python client and the well-known Cora dataset.

## Setup

We need an environment where Neo4j and GDS are available, for example AuraDS (which comes with GDS preinstalled) or Neo4j Desktop. 

Once the credentials to access this environment are available, we can install the `graphdatascience` package and import the client class.

In [ ]:
%pip install graphdatascience

In [2]:
from graphdatascience import GraphDataScience

/Users/nvitucci/github/graph-data-science-client_nvitucci/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


When using a local Neo4j setup, the default connection URI is `bolt://localhost:7687`. When using AuraDS, the connection URI is slightly different as it uses the `neo4j+s` protocol. In this case, the client should also include the `aura_ds=True` flag to enable AuraDS-recommended settings.

In [5]:
# Replace with the actual connection URI and credentials
NEO4J_CONNECTION_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "test"

gds = GraphDataScience(NEO4J_CONNECTION_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# On AuraDS:
#
# NEO4J_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
# NEO4J_USERNAME = "neo4j"
# NEO4J_PASSWORD = "..."
#
# gds = GraphDataScience(NEO4J_CONNECTION_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), aura_ds=True)

We import `json` to help in writing the Cypher queries used to load the data, and `numpy` and `pandas` for further data processing.

In [6]:
import json

import numpy as np
import pandas as pd

## Load the Cora dataset

First of all, we need to load the Cora dataset on Neo4j. The CSV files can be found at the following URIs:

In [7]:
CORA_CONTENT = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.content"
)
CORA_CITES = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.cites"
)

Upon loading, we also need to perform an additional preprocessing step to convert the `subject` field (which is a string in the dataset) into an integer, because node properties have to be numerical in order to be projected into a graph; although we could assign consecutive IDs, we assign an ID other than 0 to the first class to show how the classes are represented in the model.

We also select a number of nodes to be held out to test the model after it has been trained. **NOTE:** This is not related to the algorithm test/split ratio.

In [8]:
SUBJECT_TO_ID = {
    "Neural_Networks": 100,
    "Rule_Learning": 1,
    "Reinforcement_Learning": 2,
    "Probabilistic_Methods": 3,
    "Theory": 4,
    "Genetic_Algorithms": 5,
    "Case_Based": 6,
}

HOLDOUT_NODES = 10

We can now load the CSV files using the `LOAD CSV` statement and some basic data transformation:

In [9]:
# Define a string representation of the SUBJECT_TO_ID map using backticks
subject_map = json.dumps(SUBJECT_TO_ID).replace('"', "`")

# Cypher command to load the nodes using `LOAD CSV`, taking care of
# converting the string `subject` field into an integer and
# replacing the node label for the holdout nodes
load_nodes = f"""
    LOAD CSV FROM "{CORA_CONTENT}" AS row
    WITH 
      {subject_map} AS subject_to_id,
      toInteger(row[0]) AS extId, 
      row[1] AS subject, 
      toIntegerList(row[2..]) AS features
    MERGE (p:Paper {{extId: extId, subject: subject_to_id[subject], features: features}})
    WITH p LIMIT {HOLDOUT_NODES}
    REMOVE p:Paper
    SET p:UnclassifiedPaper
"""

# Cypher command to load the relationships using `LOAD CSV`
load_relationships = f"""
    LOAD CSV FROM "{CORA_CITES}" AS row
    MATCH (n), (m) 
    WHERE n.extId = toInteger(row[0]) AND m.extId = toInteger(row[1])
    MERGE (n)-[:CITES]->(m)
"""

# Load nodes and relationships on Neo4j
gds.run_cypher(load_nodes)
gds.run_cypher(load_relationships)

""


With the data loaded on Neo4j, we can now project a graph including all the nodes and the `CITES` relationship as undirected (and with `SINGLE` aggregation, to skip repeated relationships as a result of adding the inverse direction).

In [10]:
# Create the projected graph containing both classified and unclassified nodes
G, _ = gds.graph.project(
    "cora-graph",
    {"Paper": {"properties": ["features", "subject"]}, "UnclassifiedPaper": {"properties": ["features"]}},
    {"CITES": {"orientation": "UNDIRECTED", "aggregation": "SINGLE"}},
)

Loading: 100%|██████████| 100.0/100 [00:01<00:00, 76.26%/s] 


Then we can check the number of nodes in the newly-projected graph (which should be 2708), just to make sure it has been created correctly:

In [23]:
assert G.node_count() == 2708

## Pipeline catalog basics

Once the dataset has been loaded, we can define a node classification machine learning pipeline.

In [11]:
# Create the pipeline
node_pipeline, _ = gds.beta.pipeline.nodeClassification.create("cora-pipeline")

We can check that the pipeline has actually been created with the `list` method:

In [12]:
# List all pipelines
gds.beta.pipeline.list()

# Get the details of a specific pipeline object
gds.beta.pipeline.list(node_pipeline)

,pipelineInfo,pipelineName,pipelineType,creationTime
0,"{'featurePipeline': {'nodePropertySteps': [], ...",cora-pipeline,Node classification training pipeline,2022-12-08T23:02:30.493506000+00:00


## Configure the pipeline

We can now configure the pipeline. As a reminder, we need to:

1. Select a subset of the available node properties to be used as features for the machine learning model
1. Configure the train/test split and the number of folds for k-fold cross-validation _(optional)_
1. Configure the candidate models for training

In [13]:
# "Mark" some node properties that will be used as features
node_pipeline.selectFeatures(["features"])

name                                                     cora-pipeline
nodePropertySteps                                                   []
featureProperties                                           [features]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

In [14]:
# If needed, change the train/test split ratio and the number of folds
# for k-fold cross-validation
node_pipeline.configureSplit(testFraction=0.2, validationFolds=5)

name                                                     cora-pipeline
nodePropertySteps                                                   []
featureProperties                                           [features]
splitConfig                {'testFraction': 0.2, 'validationFolds': 5}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

Here we use Logistic Regression as an example for the training, but other algorithms (such as Random Forest) are available as well.

Some hyperparameters such as `penalty` can be single values or ranges. If they are expressed as ranges, auto-tuning is used to search their best value.

In [15]:
# Add a model candidate to train
node_pipeline.addLogisticRegression(maxEpochs=1000, penalty=(0.00038, 0.00042))

name                                                     cora-pipeline
nodePropertySteps                                                   []
featureProperties                                           [features]
splitConfig                {'testFraction': 0.2, 'validationFolds': 5}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

## Run the training

It is now possible to train the configured models. We also run a training estimate, to make sure there are enough resources to run the actual training afterwards.

The Node Classification model supports several evaluation metrics. Here we use the global metric `F1_WEIGHTED`.

**NOTE:** The `concurrency` parameter is explicitly set to 4 (the default value) for demonstration purposes. 
The maximum concurrency in the library is limited to 4 for Neo4j Community Edition.

In [17]:
# Estimate the resources needed for training the model
node_pipeline.train_estimate(
    G,
    targetNodeLabels=["Paper"],
    modelName="cora-pipeline-model",
    targetProperty="subject",
    metrics=["F1_WEIGHTED"],
    randomSeed=42,
    concurrency=4,
)

requiredMemory                                     [64 MiB ... 64 MiB]
treeView             Memory Estimation: [64 MiB ... 64 MiB]\n|-- al...
mapView              {'components': [{'components': [{'components':...
bytesMin                                                      67610384
bytesMax                                                      67642344
nodeCount                                                         2698
relationshipCount                                                10502
heapPercentageMin                                                  0.1
heapPercentageMax                                                  0.1
Name: 0, dtype: object

In [19]:
# Perform the actual training
model, stats = node_pipeline.train(
    G,
    targetNodeLabels=["Paper"],
    modelName="cora-pipeline-model",
    targetProperty="subject",
    metrics=["F1_WEIGHTED"],
    randomSeed=42,
    concurrency=4,
)

Node Classification Train Pipeline: 100%|██████████| 100.0/100 [09:56<00:00,  5.97s/%]


We can inspect the result of the training, for example to print the evaluation metrics of the trained model.

In [24]:
# Uncomment to print all stats
# print(stats.to_json(indent=2))

# Print F1_WEIGHTED metric
stats["modelInfo"]["metrics"]["F1_WEIGHTED"]["test"]

0.7367307528092472

## Use the model for prediction

After the model has been trained, it is possible to use it to classify unclassified data. 

One simple way to use the `predict` mode is to just stream the result of the prediction. This can be impractical when a graph is very large, so it should be used only for experimentation purposes.

In this example we use the `nodeLabels=["UnclassifiedPaper"]` filter to only run prediction on the unclassified nodes. It must be noted that, when using models that have `nodePropertySteps` that use relationships (such as FastRP and other models that create embeddings, but also algorithms like PageRank), the `nodeLabels` filter should not be used because it would "cut out" all the linked nodes that have a different label. We will see an example of this in the following section.

In [26]:
predicted = model.predict_stream(
    G, modelName="cora-pipeline-model", includePredictedProbabilities=True, targetNodeLabels=["UnclassifiedPaper"]
)

The result of the prediction is a DataFrame containing the predicted class and the predicted probabilities for all the classes for each node.

In [27]:
predicted

,nodeId,predictedClass,predictedProbabilities
0,0,100,"[0.01361877434161338, 0.0008177054013954495, 0..."
1,1,1,"[0.31864827446240307, 0.026098761297171216, 0...."
2,2,2,"[0.007710767899246513, 0.766314905140325, 0.00..."
3,3,2,"[0.0035000391022356513, 0.9768116873834237, 0...."
4,4,3,"[0.030847881084031987, 0.0013559744680570276, ..."
5,5,1,"[0.27506093698192285, 0.2660508673427692, 0.09..."
6,6,6,"[0.017773776963205926, 0.0607078611969441, 0.0..."
7,7,100,"[0.0028360075434722665, 0.004250369599202613, ..."
8,8,100,"[0.018410346867102398, 0.026196141169875405, 0..."
9,9,4,"[0.0115155848742497, 0.3317480405930409, 0.002..."


The order of the classes in the `predictedProbabilities` field is given in the model information, and can be used to retrieve the predicted probability for the predicted class.

In [28]:
classes = stats["modelInfo"]["classes"]
print(classes)

[1, 2, 3, 4, 5, 6, 100]


In [29]:
# Calculate the confidence percentage for the predicted class
predicted["confidence"] = predicted.apply(
    lambda row: np.floor(row["predictedProbabilities"][classes.index(row["predictedClass"])] * 100), axis=1
)

In [30]:
pd.set_option("display.precision", 6)

predicted

,nodeId,predictedClass,predictedProbabilities,confidence
0,0,100,"[0.01361877434161338, 0.0008177054013954495, 0...",73.0
1,1,1,"[0.31864827446240307, 0.026098761297171216, 0....",31.0
2,2,2,"[0.007710767899246513, 0.766314905140325, 0.00...",76.0
3,3,2,"[0.0035000391022356513, 0.9768116873834237, 0....",97.0
4,4,3,"[0.030847881084031987, 0.0013559744680570276, ...",86.0
5,5,1,"[0.27506093698192285, 0.2660508673427692, 0.09...",27.0
6,6,6,"[0.017773776963205926, 0.0607078611969441, 0.0...",82.0
7,7,100,"[0.0028360075434722665, 0.004250369599202613, ...",89.0
8,8,100,"[0.018410346867102398, 0.026196141169875405, 0...",54.0
9,9,4,"[0.0115155848742497, 0.3317480405930409, 0.002...",46.0


## Adding a data preprocessing step

The performance of the model can potentially be increased by adding more features or by using different features altogether. One way is to use models that create embeddings based on both node properties and graph features. One of such models is FastRP, which can be added via the `addNodeProperty` pipeline method.

More embedding methods are available in GDS, as well as other pre-processing algorithms.

In [31]:
node_pipeline_fastrp, _ = gds.beta.pipeline.nodeClassification.create("cora-pipeline-fastrp")

# Add a step in the pipeline that mutates the graph
node_pipeline_fastrp.addNodeProperty(
    "fastRP",
    mutateProperty="embedding",
    embeddingDimension=512,
    propertyRatio=1.0,
    randomSeed=42,
    featureProperties=["features"],
)

name                                              cora-pipeline-fastrp
nodePropertySteps    [{'name': 'gds.fastRP.mutate', 'config': {'ran...
featureProperties                                                   []
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

With the node embeddings available as features, we no longer use the original raw `features`.

In [32]:
node_pipeline_fastrp.selectFeatures(["embedding"])

name                                              cora-pipeline-fastrp
nodePropertySteps    [{'name': 'gds.fastRP.mutate', 'config': {'ran...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

In [33]:
# Configure the pipeline as before
node_pipeline_fastrp.configureSplit(testFraction=0.2, validationFolds=5)

node_pipeline_fastrp.addLogisticRegression(maxEpochs=1000, penalty=(0.00048, 0.00050))

name                                              cora-pipeline-fastrp
nodePropertySteps    [{'name': 'gds.fastRP.mutate', 'config': {'ran...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.2, 'validationFolds': 5}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

In [35]:
# Perform the actual training
model_fastrp, stats_fastrp = node_pipeline_fastrp.train(
    G,
    targetNodeLabels=["Paper"],
    modelName="cora-pipeline-model-fastrp",
    targetProperty="subject",
    metrics=["F1_WEIGHTED"],
    randomSeed=42,
    concurrency=4,
)

Node Classification Train Pipeline: 100%|██████████| 100.0/100 [01:33<00:00,  1.07%/s]


In [36]:
print(stats_fastrp["modelInfo"]["metrics"]["F1_WEIGHTED"]["test"])

0.8466258544158551


## Use the new model for prediction

Here we are **not** using the `nodeLabels=["UnclassifiedPaper"]` parameter, because FastRP depends on neighbour nodes. When using models that have `nodePropertySteps` that use relationships (such as FastRP and other models that create embeddings, but also algorithms like PageRank), the `nodeLabels` filter should not be used because it would "cut out" all the linked nodes that have a different label.

In [37]:
predicted_fastrp = model_fastrp.predict_stream(
    G, modelName="cora-pipeline-model-fastrp", includePredictedProbabilities=True
)

Node Classification Predict Pipeline: 100%|██████████| 100.0/100 [00:00<00:00, 161.51%/s]


In [43]:
predicted_fastrp.count()

nodeId                    2698
predictedClass            2698
predictedProbabilities    2698
dtype: int64

Since we have used no filters, the `predicted` result contains _all_ the nodes. The way to filter the nodes is via the `streamNodeProperty` method, which can be used only after the new property is written to the graph via the `mutate` mode. 

Instead of streaming the results, the prediction can be run in `mutate` mode to be more performant. The predicted nodes can be retrieved using the `streamNodeProperty` method with the `UnclassifiedPaper` class.

In [44]:
model_fastrp.predict_mutate(
    G,
    mutateProperty="predictedClass",
    modelName="cora-pipeline-model-fastrp",
    predictedProbabilityProperty="predictedProbabilities",
)

ClientError: Failed to invoke procedure `gds.beta.pipeline.nodeClassification.predict.mutate`: Caused by: java.lang.IllegalArgumentException: Multiple errors in configuration arguments:
				Node property `predictedProbabilities` already exists in the in-memory graph.
				Node property `predictedClass` already exists in the in-memory graph.

In [45]:
predicted_fastrp = gds.graph.streamNodeProperty(G, "predictedClass", ["UnclassifiedPaper"])

predicted_fastrp

ClientError: Failed to invoke procedure `gds.graph.streamNodeProperty`: Caused by: java.lang.IllegalArgumentException: Expecting all specified node projections to have all given properties defined. Could not find property key(s) ['predictedClass'] for label UnclassifiedPaper. Defined keys: ['features'].

In [33]:
# Retrieve node information from Neo4j using the node IDs from the prediction result
nodes = gds.util.asNodes(predicted_fastrp.nodeId.to_list())

# Create a new DataFrame containing node IDs along with node properties
nodes_df = pd.DataFrame([(node.id, node["subject"]) for node in nodes], columns=["nodeId", "subject"])

# Merge with the prediction result on node IDs, to check the predicted value
# against the original subject
#
# NOTE: This could also be replaced by just appending `node["subject"]` as a
# Series since the node order would not change, but a proper merge (or join)
# is clearer and less prone to errors.
predicted_fastrp.merge(nodes_df, on="nodeId")

,nodeId,propertyValue,subject
0,0,100,100
1,1,1,1
2,2,2,2
3,3,2,2
4,4,3,3
5,5,3,3
6,6,4,4
7,7,100,100
8,8,100,100
9,9,4,4


As we can see, the prediction for all the holdout nodes is accurate.

## Write result back to Neo4j

In [34]:
model_fastrp.predict_write(
    G,
    writeProperty="predictedSubject",
    modelName="cora-pipeline-model-fastrp",
    predictedProbabilityProperty="predictedProbabilities",
)

Node Classification Predict Pipeline: 100%|██████████| 100.0/100 [00:00<00:00, 134.49%/s]


nodePropertiesWritten                                                 5416
writeMillis                                                            199
postProcessingMillis                                                     0
preProcessingMillis                                                      0
computeMillis                                                          948
configuration            {'graphName': 'cora-graph', 'jobId': '5453396e...
Name: 0, dtype: object

## Cleanup

When the graph, the model and the pipeline are no longer needed, they should be dropped to free up memory:

In [ ]:
model.drop()
model_fastrp.drop()
node_pipeline.drop()
node_pipeline_fastrp.drop()

G.drop()
gds.run_cypher("MATCH (n) DETACH DELETE n")

It is good practice to close the client as well:

In [68]:
gds.close()